# SLICEop
Operational forecast of the St. Lawrence river's freeze-up date near Montreal, QC, Canada.  
This is the operational version of [SLICE](https://github.com/McGill-sea-ice/SLICE) the forecast developped and tested by Amélie Bouchat.

---

# Documentation and Usage

## Method

We use time series of September total cloud cover, November snowfall and December 2 m – air temperature from the [ECMWF's seasonal forecast system](https://www.ecmwf.int/en/forecasts/documentation-and-support/long-range)
to perform a multiple linear regression that predicts the date on which the water temperature near Montreal drops to the freezing point and ice formation begins. 

<div>
    <img src="Fig01.png" width=800/>
</div>  

*Fig. 1: Schematic of the regression model.*

$\text{FUD}(y) = a_{0} + a_{1}x_{1}(y) + a_{2}x_{2}(y) + a_{3}x_{3}(y)$,  
where $x_{1}$ is the September cloud cover, $x_{2}$ the November snowfall, $x_{3}$ the December 2m-air temperature and we try to find the coefficients $a_{0}$, $a_{1}$, $a_{2}$, $a_{3}$ that best predict $\text{FUD}$. Once the coefficients are found based on previous years we can forecast the $\text{FUD}$ of the current season by putting in some predictor variables $x_{1}$, $x_{2}$, $x_{3}$ for the current season.

The predictor variables are extracted from the [SEAS5.1 seasonal forecast](https://www.ecmwf.int/en/elibrary/80921-seas5-and-future-evolution-long-range-forecast-system) starting in July of each year. Once data for a given predictor variables becomes available from ERA5, the SEAS5.1 forecast data is replaced by the equivalent [ERA5 reanalysis](https://www.ecmwf.int/en/forecasts/dataset/ecmwf-reanalysis-v5) data: *e.g.* in mid October, the total cloud cover for September is available from ERA5 and is used to do the forecast instead of the forecasted total cloud cover.  
The water temperature and actual date of freeze-up are observed at the [Longueuil water treatment plant](https://www.longueuil.quebec/fr/eau-potable) near Montreal. The river is assumed to start freezing up when the water temperature at the water treatment plant falls below 0.75$^{\circ}$C, which has been shown to correlate best with observed freeze-up dates from other sources like the [St-Lawrence Seaway Management Corporation (SLSMC)](https://greatlakes-seaway.com/en/) or ice charts based on reconnaissance flights by the [Canadian Coast Guard](https://www.ccg-gcc.gc.ca/index-eng.html). 
The three predictor variables have been found to best predict the freeze-up date based on [extensive testing by Amélie Bouchat](https://github.com/McGill-sea-ice/SLICE).
They also tested the use of machine learning for the forecast, which was found to perform worse on medium to seasonal time scales.

### Automated tasks

#### Daily

1. [**`daily_Twater.py`**](https://github.com/McGill-sea-ice/SLICEop/blob/main/SLICEop/downloads/daily_Twater.py) Water temperature is tranferred from the water treatment plant daily with a temporal resolution of one minute. The daily average is computed and added to the time series (`Twater_Longueuil_updated.nc`) after a basic quality control to remove outliers that are either warmer than 30$^{\circ}$C or exhibit a spike in temperature (defined as a temperature change of more than 1$^{\circ}$C per hour). 
2. [**`daily_MODIS.py`**](https://github.com/McGill-sea-ice/SLICEop/blob/main/SLICEop/downloads/daily_MODIS.py)The most recent Terra/MODIS True Color satellite image for the greater Montreal region is downloaded from [NASA worldview](https://worldview.earthdata.nasa.gov).

<div>
    <img src="https://wvs.earthdata.nasa.gov/api/v1/snapshot?REQUEST=GetSnapshot&TIME=2025-01-20T00:00:00Z&BBOX=45.1855,-74.2417,45.8297,-73.261&CRS=EPSG:4326&LAYERS=MODIS_Terra_CorrectedReflectance_TrueColor,Coastlines_15m&WRAP=day,x&FORMAT=image/jpeg&WIDTH=893&HEIGHT=586&colormaps=,&ts=1744122633289" width="500"/>
</div>

*Fig. 2: Example of the Terra/MODIS image.*

#### Monthly

1. [**`monthly_SEAS51_ERA5.py`**](https://github.com/McGill-sea-ice/SLICEop/blob/main/SLICEop/downloads/monthly_SEAS51_ERA5.py) The ECMWF's SEAS5.1 seasonal forecast is run every 1st of the month and the data made available on the 5th. Starting in July, we try to download the desired variables on the 7th (to allow for a small buffer of 2 days in case of delays). Once past the month of a specific variable (September for total cloud cover etc.), we download the ECMWF's ERA5 reanalysis data instead to benefit from the actual observations incorporated into ERA5.
2. [**`monthly_preprocess.py`**](https://github.com/McGill-sea-ice/SLICEop/blob/main/SLICEop/prepro/monthly_preprocess.py) The preprocessing consists mainly of the computation of monthly averages (September total cloud cover, December 2 m – air temperature) and sums (November snowfall). For SEAS5.1 the preprocessing is done for each ensemble member and the ensemble mean.
3. [**`monthly_forecast.py`**](https://github.com/McGill-sea-ice/SLICEop/blob/main/SLICEop/auto/monthly_forecast.py) The regression model built from the yearly time series of the predictor variables is used with the monthly averages/sums from SEAS5.1 and/or ERA5 in order to predict the freeze-up date. If SEAS5.1 was used to compute the monthly averages/sums, the forecast is performed for each ensemble member and the ensemble mean.

#### Weekly

<div>
    <img src="Fig03.png" width=800/>
</div>  

*Fig. 3: Schematic timeline of the dates the forecast is performed. Each orange x indicates the monthly forecast, each vertical black line indicates the weekly forecast. In the black boxes is indicated for each monthly forecast whether each of the input variables originate from SEAS5.1 or ERA5.*

1. [**`weekly_ERA5.py`**](https://github.com/McGill-sea-ice/SLICEop/blob/main/SLICEop/downloads/weekly_ERA5.py) Each Monday during the months of September, November, and December and up to the 6th of the following month, we check whether ERA5 data is available to replace the SEAS5.1 data and if yes, download the respective variable.
2. [**`weekly_preprocess.py`**](https://github.com/McGill-sea-ice/SLICEop/blob/main/SLICEop/prepro/weekly_preprocess.py) If ERA5 data was downloaded in the step before, here we compute a monthly average or sum based on the combination of ERA5 and SEAS5.1 data. The preprocessing is performed for each ensemble member and the ensemble mean of SEAS5.1. If applicable, the SEAS5.1 data in each of the ensemble members is replaced by the same ERA5 data. 
3. [**`weekly_forecast.py`**](https://github.com/McGill-sea-ice/SLICEop/blob/main/SLICEop/auto/weekly_forecast.py) The forecast is performed exactly the same way as before for the monthly forecast.

<div>
    <img src="Fig04.png" width=1100/>
</div>  

*Fig. 4: Examples of how SEAS5.1 data is replaced by ERA5 data before computing the monthly average or sum during the weekly preprocessing.*

#### Yearly

1. [**`yearly_preprocess.py`**](https://github.com/McGill-sea-ice/SLICEop/blob/main/SLICEop/prepro/yearly_preprocess.py) Once a year in June, the time series shown in Fig. 1 are updated with the previous year's values: The monthly averages for September total cloud cover and December 2 m -- air temperature, the monthly sum for November snowfall and the freeze-up date calculated from the water temperature at Longueuil.

---

## Usage

### Preparation

After cloning the repository

```
git clone git@github.com:McGill-sea-ice/SLICEop.git
```

the two main requirements are

1. having an account with [Copernicus](https://www.copernicus.eu/en) to access the [ECMWF](https://www.ecmwf.int/)'s data through their [Climate Data Store](https://cds.climate.copernicus.eu/), to be used by the python module `cdsapi`. Follow the instructions under `1. Setup the CDS API personal access token` on [https://cds.climate.copernicus.eu/how-to-api](https://cds.climate.copernicus.eu/how-to-api) and then
2. create a python environment with the required packages specified in [`SLICEop/environment.yml`](https://github.com/McGill-sea-ice/SLICEop/blob/main/environment.yml), e.g. with
   
    ```
    conda env create -f environment.yml
    ```

    If you choose another way to create the environment with conda, make sure that the environment is called `sliceop`. If you create the environment in another way, like `virtualenv`,     make sure the environment is loaded by changing the lines under `#load conda environment` in `SLICEop/SLICEop/auto/run_*.sh`.

Further make sure to update the paths in [`SLICEop/setup.sh`](https://github.com/McGill-sea-ice/SLICEop/blob/main/setup.sh) and run `source setup.sh` to set the environment variables.

The program will also create data that is compatible with javascript which will be used to create an interactive [echarts](https://echarts.apache.org) figure in [`SLICEop/SLICEop/echart/sliceop.html`](https://github.com/McGill-sea-ice/SLICEop/blob/main/SLICEop/echart/sliceop.html). For this to function [`echarts`](https://github.com/apache/echarts/blob/5.6.0/dist/echarts.js) (version 5.6.0) needs to be downloaded and placed in the directory `SLICEop/SLICEop/echart/` as `echarts.js`.

### Initialization

The initialization includes a large number of downloads from the Climate Data Store and can take a long time (on the order of days).  

Run 

```
source setup.sh
```

if not already done and then run

```
SLICEop/SLICEop/init.sh
```

[`init.sh`](https://github.com/McGill-sea-ice/SLICEop/blob/main/SLICEop/init.sh) will create a couple of files that are needed later on, then call [`SLICEop/SLICEop/downloads/initial_download_ERA5.py`](https://github.com/McGill-sea-ice/SLICEop/blob/main/SLICEop/downloads/initial_download_ERA5.py) to download the ERA5 data required from previous years. Depending on how busy the Climate Data Store's server is, this could take a long time. The time period and region downloaded are all specified in `initial_download_ERA5.py`.
Following, `init.sh` will call [`SLICEop/SLICEop/downloads/initial_Twater.py`](https://github.com/McGill-sea-ice/SLICEop/blob/main/SLICEop/downloads/initial_Twater.py) do initialize the time series of water temperature from the Longueuil water treatment plant. Additionally, the time series of freeze-up dates is computed here. This script is highly specialized to work with this specific time series as there are gaps to fill, different data formats to account for etc. If you want to use a different time series of water temperature, this script will be of little use.  
Lastly, `init.sh` will run [`SLICEop/SLICEop/prepro/yearly_preprocess.py`](https://github.com/McGill-sea-ice/SLICEop/blob/main/SLICEop/prepro/yearly_preprocess.py) as if it were the last June in order to get an initial file of monthly predictors from previous years.



### Testing

Once the initialization done, there is the possibility to run a test that simulates the weekly and monthly forecasts for the period of past years specified in [`SLICEop/SLICEop/run_test.sh`](https://github.com/McGill-sea-ice/SLICEop/blob/main/SLICEop/run_test.sh).  

Running this script will generate two files per simulated forecast season starting in year `YYYY`, one (`YYYYFUDmonthly`) with all the monthly forecasts and one (`YYYYFUDweekly`) with all the weekly forecasts.  
The generated files will have the format:
```
time,number,FUD
1997-07-07,0,359
1997-07-07,1,353
1997-07-07,2,346
...
```
The first column (`time`) represents the date the forecast was run. The second column `number` is the number of the ensemble member, where `0` represents either the ensemble mean or is the only number present when there is no ensemble (ERA5 data as input to the forecast). The third column (`FUD`) is the predicted freeze-up day of the year, where all years are assumed to have 365 years, e.g. December 22 is always `FUD = 356`. `FUD > 365` represent a freeze-up date in the next year on day `FUD - 365`.  
No figures will be generated during the test.

### Operational use

Once the initilization and testing were successful the forecast can go into operational use. To have things running automatically, we need to add the calls to the functions [`run_daily.sh`](https://github.com/McGill-sea-ice/SLICEop/blob/main/SLICEop/auto/run_daily.sh), [`run_weekly.sh`](https://github.com/McGill-sea-ice/SLICEop/blob/main/SLICEop/auto/run_weekly.sh), [`run_monthly.sh`](https://github.com/McGill-sea-ice/SLICEop/blob/main/SLICEop/auto/run_monthly.sh) and [`run_yearly.sh`](https://github.com/McGill-sea-ice/SLICEop/blob/main/SLICEop/auto/run_yearly.sh) to a crontab so cron can run them automatically. An example of how the crontab could look like is given in [`SLICEop/to_crontab`](https://github.com/McGill-sea-ice/SLICEop/blob/main/to_crontab).

```
# run daily script at 10AM
0 10 * * * ${local_path}/auto/run_daily.sh &>> ${local_path}/auto/run_daily.log

# run weekly script every Monday at 9AM
# 0 9 * * 1 ${local_path}/auto/run_weekly.sh &>> ${local_path}/auto/run_weekly.log

# run monthly script at 10:30AM  on the 7th of each month
30 10 7 * * ${local_path}/auto/run_monthly.sh &>> ${local_path}/auto/run_monthly.log

# run yearly script at noon on the 10th of June
0 12 10 6 * ${local_path}/auto/run_yearly.sh &>> ${local_path}/auto/run_yearly.log
```

Note that the specific hours when the different scripts are being run are unimportant and the weekly script can be run any other day of the week as well. However, the monthly script `run_monthly.sh` **must** be run on the 7th of each month!   
It is also adviable to run the monthly script **after** the weekly script. In case they are executed on the same day (the 7th being a Monday in the example), this ensures that the full previous month of data is taken from ERA5 (in `run_monthly.sh`) and not a combination of SEAS5.1 and ERA5 (as in `run_weekly.sh`).

#### **[`run_daily.sh`](https://github.com/McGill-sea-ice/SLICEop/blob/main/SLICEop/auto/run_daily.sh)**

`run_daily.sh` runs four scripts: [`daily_Twater.py`](https://github.com/McGill-sea-ice/SLICEop/blob/main/SLICEop/downloads/daily_Twater.py), [`daily_MODIS.py`](https://github.com/McGill-sea-ice/SLICEop/blob/main/SLICEop/downloads/daily_MODIS.py), [`daily_plots.py`](https://github.com/McGill-sea-ice/SLICEop/blob/main/SLICEop/auto/daily_plots.py) and [`daily_prepare_data_for_echart.py`](https://github.com/McGill-sea-ice/SLICEop/blob/main/SLICEop/auto/daily_prepare_data_for_echart.py).  

1. [`daily_Twater.py`](https://github.com/McGill-sea-ice/SLICEop/blob/main/SLICEop/downloads/daily_Twater.py) adds the temperature data $T_{water}$ from the Longueuil water treatment plant from the previous day to the existing timeseries (`Twater_Longueuil_updated.nc`) after removing outliers. $T_{water}$ from the water treatment plant has a temporal resolution of one minute. In this 1-min timeseries, outliers are flagged when $T_{water}$ changes more than $1^{\circ}C/$hour, is larger than $30^{\circ}$C or deviates more than $3^{\circ}$C from the climatological average of its day. Outliers are then replaced by linearly interpolated valid values and the 24 hours of 1-min data are then averaged to add the daily $T_{water}$ to `Twater_Longueuil_updated.nc`.  

2. [`daily_MODIS.py`](https://github.com/McGill-sea-ice/SLICEop/blob/main/SLICEop/downloads/daily_MODIS.py) downloads a Terra/MODIS satellite image of an area around Montreal for the previous day from [NASA worldview](https://worldview.earthdata.nasa.gov/?v=-74.39960157232706,45.18549999999999,-73.10309842767295,45.8297&t=2025-02-09-T18%3A11%3A43Z).  

3. [`daily_plots.py`](https://github.com/McGill-sea-ice/SLICEop/blob/main/SLICEop/auto/daily_plots.py) plots the updated $T_{water}$ timeseries to `Twater.png`:

<div>
    <img src="Fig05.png" width=500/>
</div>  

*Fig. 5: Example figure for the evolution of $T_{water}$ in the current forecast season. The blue line indicates the observed $T_{water}$, the dashed gray line represents the climatological seasonal cycle and the vertical dashed red line marks the observed freeze-up date of the season.*  

4. [`daily_prepare_data_for_echart.py`](https://github.com/McGill-sea-ice/SLICEop/blob/main/SLICEop/auto/daily_prepare_data_for_echart.py) gathers all available data of past and current forecast seasons' water temperatures $T_{water}$ and freeze-up dates $\text{FUD}$ computes a climatlogy etc. and saves everything into `json` format to be used in generating an interactive [echarts](https://echarts.apache.org/en/index.html) figure. This includes a [cmocean](https://matplotlib.org/cmocean/) colormap converted in hex color codes to be used in the chart. If you wish to change the colormap of the echarts figure, it is defined as `cmap` at the beginning of `daily_prepare_data_for_echart.py`. An additional file called `latest.json` is created which includes `latestForecastIssued`, the date the latest forecast was made and `latestForecast`, the freeze-up date that this forecast predicted. **Note** that for testing purposes, the `json` data is converted into javascript and read in by `sliceop.html` as scripts and not as data to avoid having to set up a html server to load the data. The conversion is done in `echart_cheat.sh`.

#### **[`run_monthly.sh`](https://github.com/McGill-sea-ice/SLICEop/blob/main/SLICEop/auto/run_monthly.sh)**

`run_monthly.sh` runs four scripts: [`monthly_SEAS51_ERA5.py`](https://github.com/McGill-sea-ice/SLICEop/blob/main/SLICEop/downloads/monthly_SEAS51_ERA5.py), [`monthly_preprocess.py`](https://github.com/McGill-sea-ice/SLICEop/blob/main/SLICEop/prepro/monthly_preprocess.py), [`monthly_forecast.py`](https://github.com/McGill-sea-ice/SLICEop/blob/main/SLICEop/auto/monthly_forecast.py) and [`monthly_plots.py`](https://github.com/McGill-sea-ice/SLICEop/blob/main/SLICEop/auto/monthly_plots.py).

1. [`monthly_SEAS51_ERA5.py`](https://github.com/McGill-sea-ice/SLICEop/blob/main/SLICEop/downloads/monthly_SEAS51_ERA5.py) downloads the seasonal forecast SEAS5.1 data issued on the 1st of the current month and/or the ERA5 reanalysis data for the past month from the ECMWF's Climate Data Store. If available, ERA5 is downloaded (e.g. September total cloud cover should be available from ERA5 on October 7 and is thus downloaded instead of SEAS5.1). 

2. [`monthly_preprocess.py`](https://github.com/McGill-sea-ice/SLICEop/blob/main/SLICEop/prepro/monthly_preprocess.py) takes the data downloaded by `monthly_SEAS51_ERA5.py` and computes the monthly predictor variables. The mean for September total cloud cover and December 2m-air temperature, and the accumulated snowfall for November. The predictor variables are computed for each ensemble member and the ensemble mean of SEAS5.1. ERA5, which is not an ensemble, is treated as an ensemble mean without any members. The preprocessed predictor variables are written into `SLICEop/SLICEop/prepro/input_forecast.nc`. This file is overwritten every month, it is thus not a long-term storage of the monthly predictor variables!

3. [`monthly_forecast.py`](https://github.com/McGill-sea-ice/SLICEop/blob/main/SLICEop/auto/monthly_forecast.py) will perform the monthly forecast based on the coefficients found with the predcitor variables and freeze-up dates from previous years in `monthly_predictors.nc` (see [Method](#Method)), using the variables for the current season in `input_forecast.nc` to predict the freeze-up date of the current season. The forecast is done for each ensemble member and the ensemble mean if SEAS5.1 data is included in `input_forecast.nc`, or only for the "ensemble mean" if only ERA5 data is found in `input_forecast.nc` (ERA5 is not an ensemble but its output is treated as an ensemble mean here).
The forecasted freeze-up dates are written into file `YYYYFUDmonthly`, where `YYYY` is the year in which the current forecast season started and the files have the format
```
time,number,FUD
1997-07-07,0,359
1997-07-07,1,353
1997-07-07,2,346
...
```
    The first column (`time`) represents the date the forecast was run. The second column `number` is the number of the ensemble member, where `0` represents either the ensemble mean or is the only number present when there is no ensemble (ERA5 data as input to the forecast). The third column (`FUD`) is the predicted freeze-up day of the year, where all years are assumed to have 365 years, e.g. December 22 is always `FUD = 356`. `FUD > 365` represent a freeze-up date in the next year on day `FUD - 365`.  

4. [`monthly_plots.py`](https://github.com/McGill-sea-ice/SLICEop/blob/main/SLICEop/auto/monthly_plots.py) updates `forecast.jpg`, a figure showing the evolution of the forecasted freeze-up dates with progressing time in the current forecast season.
    
<div>
    <img src="Fig06.jpg" width=600/>
</div> 

*Fig. 6: Example figure for the evolution the forecasted freeze-up dates in the current forecast season. The purple line indicates the forecasted freeze-up date based on the ensemble mean with the orange crosses marking the dates a new monthly forecast is issued based on a new SEAS5.1 seasonal forecast. The weekly and monthly forecasts based on infividual ensemble members are represented by gray markers to give an approximation of the ensemble spread.*

#### **[`run_weekly.sh`](https://github.com/McGill-sea-ice/SLICEop/blob/main/SLICEop/auto/run_weekly.sh)**

`run_weekly.sh` runs four scripts: [`weekly_ERA5.py`](https://github.com/McGill-sea-ice/SLICEop/blob/main/SLICEop/downloads/weekly_ERA5.py), [`weekly_preprocess.py`](https://github.com/McGill-sea-ice/SLICEop/blob/main/SLICEop/prepro/weekly_preprocess.py), [`weekly_forecast.py`](https://github.com/McGill-sea-ice/SLICEop/blob/main/SLICEop/auto/weekly_forecast.py) and [`weekly_plots.py`](https://github.com/McGill-sea-ice/SLICEop/blob/main/SLICEop/auto/weekly_plots.py).

1. [`weekly_ERA5.py`](https://github.com/McGill-sea-ice/SLICEop/blob/main/SLICEop/downloads/weekly_ERA5.py) downloads available ERA5 data of the current month to replace the SEAS5.1 seasonal forecast data when possible (see [description in Method:Automated tasks:Weekly](#Weekly)).

2. [`weekly_preprocess.py`](https://github.com/McGill-sea-ice/SLICEop/blob/main/SLICEop/prepro/weekly_preprocess.py) does the preprocessing as in `monthly_preprocess.py` but ERA5 data that was downloaded in the step before (if any) replaces the SEAS5.1 data before calculating the monthly mean or sum. This script outputs `input_forecast_weekly.nc` analogously to `input_forecast.nc` for the monthly forecast.

3. [`weekly_forecast.py`](https://github.com/McGill-sea-ice/SLICEop/blob/main/SLICEop/auto/weekly_forecast.py) runs the weekly forecast exactly like the monthly forecast but replacing the input data with input from `input_forecast_weekly.nc`. Forecasted freeze-up dates are written into file `YYYYFUDweekly`, in the same format as `YYYYFUDmonthly`.

4. [`weekly_plots.py`](https://github.com/McGill-sea-ice/SLICEop/blob/main/SLICEop/auto/weekly_plots.py) updates `forecast.jpg`, a figure showing the evolution of the forecasted freeze-up dates with progressing time in the current forecast season.

#### **[`run_yearly.sh`](https://github.com/McGill-sea-ice/SLICEop/blob/main/SLICEop/auto/run_yearly.sh)**

`run_yearly.sh` runs one script: [`yearly_preprocess.py`](https://github.com/McGill-sea-ice/SLICEop/blob/main/SLICEop/prepro/yearly_preprocess.py).

1. [`yearly_preprocess.py`](https://github.com/McGill-sea-ice/SLICEop/blob/main/SLICEop/prepro/yearly_preprocess.py) resets a couple of variables to values reflecting the start of the forecast season (*e.g.* `frozen` is set to `False` etc.) and updates the time series of predictor variables and observed freeze-up dates with the values from the past year so that they are included in the calculations of the regression coefficients.